In [ ]:
import importlib
import torch
import Data_Prep
importlib.reload(Data_Prep)
from Data_Prep import Data_Prep, Player_IO
import Prep_Map
importlib.reload(Prep_Map)
import Output_Map
importlib.reload(Output_Map)
import Player_Dataset
importlib.reload(Player_Dataset)
from Player_Dataset import Player_Dataset, Create_Test_Train_Datasets
import Player_Model
importlib.reload(Player_Model)
from Player_Model import RNN_Model
from torch.optim import lr_scheduler
import Model_Train
importlib.reload(Model_Train)
from Model_Train import trainAndGraph
from Constants import device, DEFAULT_NUM_LAYERS_HITTER, DEFAULT_HIDDEN_SIZE_HITTER

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
batch_size = 200
num_epochs = 70
num_layers = DEFAULT_NUM_LAYERS_HITTER
hidden_size = DEFAULT_HIDDEN_SIZE_HITTER

Experimental Pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.experimental_prep_map, Output_Map.base_output_map)
hitter_io_list = data_prep.Generate_IO_Hitters("WHERE lastMLBSeason<? AND signingYear<? AND isHitter=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(hitter_io_list, 0.25, 0)

In [ ]:
hitting_mutators = data_prep.Generate_Hitting_Mutators(batch_size, Player_IO.GetMaxLength(hitter_io_list))

network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, hitting_mutators, data_prep=data_prep, is_hitter=True)
network = network.to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=0.0025)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=30, cooldown=10, verbose=False)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              len(train_dataset), 
              len(test_dataset), 
              optimizer, 
              scheduler, 
              num_epochs = num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              model_name="Models/default_sqrt_hitter", 
              save_last=True, 
              elements_to_save=[0])

Base Pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.base_prep_map, Output_Map.base_output_map)
hitter_io_list = data_prep.Generate_IO_Hitters("WHERE lastMLBSeason<? AND signingYear<? AND isHitter=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(hitter_io_list, 0.25, 0)

In [ ]:
hitting_mutators = data_prep.Generate_Hitting_Mutators(batch_size, Player_IO.GetMaxLength(hitter_io_list))

num_layers = DEFAULT_NUM_LAYERS_HITTER
hidden_size = DEFAULT_HIDDEN_SIZE_HITTER
network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, hitting_mutators, data_prep=data_prep, is_hitter=True)
network = network.to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=0.0025)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=30, cooldown=10, verbose=False)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              len(train_dataset), 
              len(test_dataset), 
              optimizer, 
              scheduler, 
              num_epochs=num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              model_name="Models/default_hitter", 
              save_last=True,
              elements_to_save=[0])

Statsonly pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.statsonly_prep_map, Output_Map.base_output_map)
hitter_io_list = data_prep.Generate_IO_Hitters("WHERE lastMLBSeason<? AND signingYear<? AND isHitter=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(hitter_io_list, 0.25, 0)

In [ ]:
hitting_mutators = data_prep.Generate_Hitting_Mutators(batch_size, Player_IO.GetMaxLength(hitter_io_list))

num_layers = DEFAULT_NUM_LAYERS_HITTER
hidden_size = DEFAULT_HIDDEN_SIZE_HITTER
network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, hitting_mutators, data_prep=data_prep, is_hitter=True)
network = network.to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=0.0025)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=30, cooldown=10, verbose=False)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              len(train_dataset), 
              len(test_dataset), 
              optimizer, 
              scheduler, 
              num_epochs=num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              model_name="Models/default_statsonly_hitter", 
              save_last=True, 
              elements_to_save=[0])